# Homework 2:

## Part I:

<ol>
  <li><strong>T2V(WB.Drama,UB.Books.Genre,"Drama")</strong>.<br>The entries within WorldBooks' Drama table should be mapped into the USBooks' genre column, specifically with the value "Drama".</li>
  <br>
  <li><strong>T2T(WB.USSales,UB.Sales,-)</strong>.<br>The structure of WorldBooks' US Sales table is the same as the USBooks' Sales table.</li>
  <br>
  <li><strong>A2A(WB.USSales.total,UB.Sales.total,-)</strong>.<br>The entries within WorldBooks' total sales column in its US Sales table should be mapped directly into the USBooks' Sales total sales column without any transformations applied.</li>
  <br>
    <li><strong>V2T(WB.Stores.id,UB.Stores.id,[generated(GlobalID),UNION(WB.Stores.id,UB.Stores.id)])</strong>.<br>The id from both WorldBooks' and USBooks' tables are unioned together, where a global ID is generated from the result. The columns are stored in a new table, StoreIdMap.</li>
  <br>
    <li><strong>A2V([WB.Authors.FirstName,WB.Authors.MiddleName,WB.Authors.LastName],UB.Authors.Name, CONCAT([WB.Authors.FirstName,WB.Authors.MiddleName,WB.Authors.LastName])=UB.Authors.Name)</strong>.<br>The various name entries in WorldBooks' Authors table should be concatenated into the USBooks' Authors name column.</li>
</ol>



## Part II:

Please see the attached R script to indicate that I am sampling 300K random rows to be used in my calculations - this is to reduce overall computational complexity. I attached the sample of records actually used in the zip file. 

I chose to use R where I felt more comfortable writing an insert function that batches the rows efficiently and reduces memory required.

In [ ]:
import tempfile
import argparse
import csv
import os

import dedupe
import psycopg2
from psycopg2.extras import DictCursor

In [ ]:
KEY_FIELD = 'visitor_id'
SOURCE_TABLE = 'visitors'

FIELDS =  [{'field': 'firstname', 'variable name': 'firstname',
           'type': 'String','has missing': True},
           {'field': 'lastname', 'variable name': 'lastname',
           'type': 'String','has missing': True},
           {'field': 'uin', 'variable name': 'uin',
           'type': 'String','has missing': True},
           {'field': 'meeting_loc', 'variable name': 'meeting_loc',
           'type': 'String','has missing': True}
           ]

In [ ]:
deduper = dedupe.Dedupe(FIELDS)
from sklearn.linear_model import LogisticRegression
deduper.classifier = LogisticRegression()

In [ ]:
try:
    con = psycopg2.connect(database='',
                          host='',
                           password='',
                          cursor_factory=DictCursor)
    print ("I've connected")
except Exception as e:
    print (e)
c = con.cursor()

In [ ]:
# Check total rows
c.execute("select count(*) from visitors")
total_unique = c.fetchone()
cnt = total_unique['count']
print(cnt)

# Check distinct firstname and lastname on the same day:
c.execute("select count(*) from (select distinct firstname, lastname, apptmade::date from visitors where apptmade <> 'None') a")
unique_days = c.fetchone()
cnt2 = unique_days['count']
print(cnt2)

print("Differentiating between same day visitor names can yield:",cnt-cnt2)

I will assume now, to reduce further computation down the line, that there is an extreme level of unlikelihood that 2 separate people visited the white house on the same day, with the same full name. Already, it is highly unlikely you meet someone with the exact full name that you have - let alone on the same day in the same place.

In [ ]:
# Attempting to pre-reduce roles, issue with gathering all attributes and de-duping - yet able to save a few instead of just
# complete distinct
c.execute("select count(*) from (select a.firstname, a.lastname, b.uin, b.meeting_loc \
from \
(select distinct firstname, lastname, apptmade::date \
from visitors \
where apptmade <> 'None') a \
join \
(select distinct firstname, lastname, uin, meeting_loc, apptmade::date \
from visitors where apptmade <> 'None') b \
on a.firstname = b.firstname and a.lastname = b.lastname and a.apptmade = b.apptmade \
union \
select firstname, lastname, uin, meeting_loc \
from visitors \
where apptmade = 'None') c")
row = c.fetchone()
count = row['count']
# Keeping sample size to 10% given computational constraints
sample_size = int(count * .1)
print(count)
print(sample_size)

I elected to remove null values here for scalability purposes, especially when you consider the percentage of incomplete records in the total dataset. I am making this decision without context beyond the task at hand and believing this to be the best choice.

In [ ]:
# Modifying for complete records
print ('Generating sample of {} records'.format(sample_size))
with con.cursor('deduper') as c_deduper:
    c_deduper.execute("select a.firstname, a.lastname, b.uin, b.meeting_loc \
from \
(select distinct firstname, lastname, apptmade::date \
from visitors \
where apptmade <> 'None') a \
join \
(select distinct firstname, lastname, uin, meeting_loc, apptmade::date \
from visitors where apptmade <> 'None') b \
on a.firstname = b.firstname and a.lastname = b.lastname and a.apptmade = b.apptmade \
union \
select firstname, lastname, uin, meeting_loc \
from visitors \
where apptmade = 'None'")
    temp_d = dict((i, row) for i, row in enumerate(c_deduper))
    deduper.sample(temp_d, sample_size)

In [ ]:
print ('Begin Active Learning Portion')
dedupe.convenience.consoleLabel(deduper)

Here, the "Oracle" has determined that he is satisfied with the current matches presented and is ready to begin the model. In this case, they assumed that common nicknames applied and allowed for liberal misspellings.

In [ ]:
print('Pre Model Training')
deduper.prepare_training(temp_d, sample_size =sample_size)

In [ ]:
print ('Begin Training')
deduper.train()

In [ ]:
print ('Save training file to JSON format: {}'.format('train.json'))
with open('train.json', 'w') as training_file:
    deduper.writeTraining(training_file)

deduper.cleanupTraining()

In [ ]:
print ('Creating blocking_map table')
c.execute("""
    DROP TABLE IF EXISTS blocking_map
    """)
c.execute("""
    CREATE TABLE blocking_map
    (block_key VARCHAR(200), visitor_id INTEGER)
    """)

In [ ]:
for field in deduper.blocker.index_fields:
    print ('Selecting distinct values for "{}"'.format(field))
    c_index = con.cursor('index')
    c_index.execute("""
        SELECT DISTINCT %s FROM %s
        """ % (field, SOURCE_TABLE))
    field_data = (row[field] for row in c_index)
    deduper.blocker.index(field_data, field)
    c_index.close()

In [ ]:
print ('Generating blocking map')
c_block = con.cursor('block')
c_block.execute("""
    SELECT * FROM %s
    """ % SOURCE_TABLE)
full_data = ((row[KEY_FIELD], row) for row in c_block)
b_data = deduper.blocker(full_data)

In [ ]:
print ('Inserting blocks into blocking_map')
csv_file = tempfile.NamedTemporaryFile(prefix='blocks_', delete=False, mode ='w')
csv_writer = csv.writer(csv_file)
for x in b_data:
    csv_writer.writerow(list(x))
#csv_writer.writerows(b_data)
csv_file.close()

In [ ]:
f = open(csv_file.name, 'r')
c.copy_expert("COPY blocking_map FROM STDIN CSV", f)
f.close()

os.remove(csv_file.name)

con.commit()

In [ ]:
print ('Indexing blocks')
c.execute("""
    CREATE INDEX blocking_map_key_idx ON blocking_map (block_key)
    """)
c.execute("DROP TABLE IF EXISTS plural_key")
c.execute("DROP TABLE IF EXISTS plural_block")
c.execute("DROP TABLE IF EXISTS covered_blocks")
c.execute("DROP TABLE IF EXISTS smaller_coverage")

In [ ]:
print ('Calculating plural_key')
c.execute("""
    CREATE TABLE plural_key
    (block_key VARCHAR(200),
    block_id SERIAL PRIMARY KEY)
    """)
c.execute("""
    INSERT INTO plural_key (block_key)
    SELECT block_key FROM blocking_map
    GROUP BY block_key HAVING COUNT(*) > 1
    """)

In [ ]:
print ('Indexing block_key')
c.execute("""
    CREATE UNIQUE INDEX block_key_idx ON plural_key (block_key)
    """)

In [ ]:
print ('Calculating plural_block')
c.execute("""
    CREATE TABLE plural_block
    AS (SELECT block_id, visitor_id
    FROM blocking_map INNER JOIN plural_key
    USING (block_key))
    """)

In [ ]:
print ('Adding {} index'.format(KEY_FIELD))
c.execute("""
    CREATE INDEX plural_block_visitor_id_idx
    ON plural_block (visitor_id)
    """)
c.execute("""
    CREATE UNIQUE INDEX plural_block_block_id_visitor_id_uniq
    ON plural_block (block_id, visitor_id)
    """)

In [ ]:
print ('Creating covered_blocks')
c.execute("""
    CREATE TABLE covered_blocks AS
    (SELECT visitor_id,
    string_agg(CAST(block_id AS TEXT), ','
    ORDER BY block_id) AS sorted_ids
    FROM plural_block
    GROUP BY 1)
    """)

In [ ]:
print ('Indexing covered_blocks')
c.execute("""
    CREATE UNIQUE INDEX covered_blocks_visitor_id_idx
    ON covered_blocks (visitor_id)
    """)
print ('Committing')

In [ ]:
print ('Creating smaller_coverage')
c.execute("""
    CREATE TABLE smaller_coverage AS
    (SELECT visitor_id, block_id,
    TRIM(',' FROM split_part(sorted_ids,
    CAST(block_id AS TEXT), 1))
    AS smaller_ids
    FROM plural_block
    INNER JOIN covered_blocks
    USING (visitor_id))
    """)
con.commit()

In [ ]:
print ('Clustering...')
c_cluster = con.cursor('cluster')
c_cluster.execute("""
    SELECT *
    FROM smaller_coverage
    INNER JOIN %s
    USING (%s)
    ORDER BY (block_id)
    """ % (SOURCE_TABLE, KEY_FIELD))

In [ ]:
def candidates_gen(result_set):
    lset = set
    block_id = None
    records = []
    i = 0
    for row in result_set:
        if row['block_id'] != block_id:
            if records:
                yield records
            
            block_id = row['block_id']
            records = []
            i += 1
            
            if i % 10000 == 0:
                print ('{} blocks'.format(i))

        smaller_ids = row['smaller_ids']
        if smaller_ids:
            smaller_ids = lset(smaller_ids.split(','))
        else:
            smaller_ids = lset([])
        
        records.append((row[KEY_FIELD], row, smaller_ids))
    
    if records:
        yield records

In [ ]:
clustered_dupes = deduper.matchBlocks(candidates_gen(c_cluster), threshold=0.5)

In [ ]:
print ('Creating entity_map table')
c.execute("DROP TABLE IF EXISTS entity_map")
c.execute("""
      CREATE TABLE entity_map (
      %s INTEGER,
      canon_id INTEGER,
      cluster_score FLOAT,
      PRIMARY KEY(%s)
      )""" % (KEY_FIELD, KEY_FIELD))

In [ ]:
print ('Inserting entities into entity_map')
for cluster, scores in clustered_dupes:
    cluster_id = cluster[0]
    for key_field, score in zip(cluster, scores):
        c.execute("""
              INSERT INTO entity_map
              (%s, canon_id, cluster_score)
              VALUES (%s, %s, %s)
              """ % (KEY_FIELD, key_field, cluster_id, score))
        con.commit()

Was able to generate result sets on a sample of data, further computational power may yield the ability to test on the full data-set. The methods applied were both active and semi-supervised learning combined with some assumptions. There may be even more opportunities with more context to reduce the data-set size prior to using dedupe.

I have attached the (sample) blocks in the zip file. Improvements for the next iteration include implementing a batch insert function in python similar to the one written in R. Final entity blocks were ran for 2 hours of computation time and were subsequently cut off.